In [64]:
import os
import pandas as pd
import yaml
import copy
import random
from pathlib import Path
import matplotlib.pyplot as plt
from copy import deepcopy

import aot

In [65]:
base_dir = Path(aot.__path__[0])
core_expt_yaml_path = base_dir / "experiment/core_exp_settings.yml"
stimuli_temp_path = base_dir / "experiment/stimuli_settings_temp.yml"
core_settings = yaml.load(open(core_expt_yaml_path), Loader=yaml.FullLoader)
stimuli_settings_temp = yaml.load(open(stimuli_temp_path), Loader=yaml.FullLoader)  

settings_root_path = base_dir / core_settings["paths"]["settings_path"] 
video_db_path = base_dir / 'data/videos/database_originals.tsv'
video_db = pd.read_csv(video_db_path, sep='\t')

main_settings_dir = base_dir / core_settings["paths"]["settings_path"] / "main"
memory_settings_dir = base_dir / core_settings["paths"]["settings_path"] / "memory"
subject_number = core_settings['various']['subject_number']
run_number = core_settings['various']['run_number']
session_number = core_settings['various']['session_number']
unique_video_number_per_session = core_settings['various']['unique_video_number_per_session']
img_number_per_session = unique_video_number_per_session



unused_videos = []
for i in range(1, len(video_db)):
    if video_db['grade'][i] != 'j' and video_db['grade'][i] != 'NA': 
        unused_videos.append(video_db['video_name'][i])
print("videos number: ", len(unused_videos))

imgs_from_unused_videos = []
for i in range(len(unused_videos)):
    imgs_from_unused_videos.append(unused_videos[i].replace('.mp4', '.png'))

videos number:  528


In [66]:
def gen_memory_design(subject,session):
    print(f"subject: {subject}, session: {session}")
    #read main experiment settings
    main_runs_settings = []
    main_runs_videos = []
    main_runs_videos_name = []#raw video name without .mp4
    for ind in range(run_number):
        run = ind + 1
        runfile = f"experiment_settings_sub_{str(subject).zfill(2)}_ses_{str(session).zfill(2)}_run_{str(run).zfill(2)}.yml"
        runfile = main_settings_dir / runfile
        runsetting = yaml.load(open(runfile), Loader=yaml.FullLoader)
        main_runs_settings.append(runsetting)
    for runsetting in main_runs_settings:
        raw_video_list = runsetting["stimuli"]["movie_files"]
        video_list = [video for video in raw_video_list if video != "blank"]
        #print(video_list)
        main_runs_videos.append(video_list)
    for videos in main_runs_videos:
        name_list= [video.split(".")[0] for video in videos]
        main_runs_videos_name.append(name_list)
        #print(name_list)
    #flatten the videos list
    flat_video_names = [item for sublist in main_runs_videos_name for item in sublist]
    #remove repeated videos
    flat_video_names = list(set(flat_video_names))
    #print(flat_video_names)
    imgnames = [video + ".png" for video in flat_video_names]
    print(imgnames)
    #save imgnames as settings for memory experiment yaml file
    settings = settings = copy.deepcopy(stimuli_settings_temp)

    half_img_num = int((img_number_per_session)/2)
    #sample half_img_num images from imgnames
    imgnames_used = random.sample(imgnames, half_img_num)
    imgnames_unused = random.sample(imgs_from_unused_videos, half_img_num)
    imgnames_final = imgnames_used + imgnames_unused
    random.shuffle(imgnames)

    settings["stimuli"]["picture_files"] = imgnames_final##not right. should subsample from imgnames and add some img from unused videos
    print(settings)
    outfile = f"experiment_settings_sub_{str(subject).zfill(2)}_ses_{str(session).zfill(2)}_run_01.yml"
    outfile = memory_settings_dir / outfile
    with open(outfile, "w") as f:
        yaml.dump(settings, f)


In [67]:
def gen_memory_desing_for_all():
    for sub in range(subject_number):
        sub = sub + 1
        for ses in range(session_number):
            ses = ses + 1
            gen_memory_design(sub, ses)       

In [68]:
gen_memory_desing_for_all()

subject: 1, session: 1
['S_yt-XpM1Jtwlvvk_42.png', 'R_S_yt-l_pvPAznG1A_20.png', 'S_yt-8QCcmqaEMPI_16.png', 'S_Pexels_Videos_2795737.png', 'S_yt-2dRSJcaJIXw_32.png', 'R_S_yt-FhnT_42bV7M_11.png', 'S_yt-OEY9O2MglQM_7.png', 'S_yt-Fm97IY0Z2LU_79.png', 'R_S_yt-h09NHEfoKb0_8.png', 'R_S_yt-UzWRbs9Syf4_7.png', 'S_yt-GyMWMiWvuiA_237.png', 'R_S_yt-CLX1rmSf2Rs_194.png', 'S_yt-LUIvP_NDWAs_45.png', 'S_yt-RKo978aBdzw_173.png', 'R_S_yt-QI6k6mhpxLo_453.png', 'R_S_yt-CJeyOFsd3lg_80.png', 'R_S_yt-v9WGfC67DPk_50.png', 'S_yt-2i53vKte7SA_117.png', 'S_ILSVRC2015_train_00595000.png', 'S_production_ID_3709168.png', 'S_yt-lGQ3wMI5-Yg_140.png', 'S_ILSVRC2015_train_00379000.png', 'R_S_yt-5ws2gxbcjs4_381.png', 'S_ILSVRC2015_train_00743006.png', 'S_yt-2y6A9DJPsDU_65.png', 'S_yt-0Zze3Ff12Ys_58.png', 'R_S_yt-8QCcmqaEMPI_16.png', 'S_yt-PltpRRYmQQk_62.png', 'S_yt-7XDVHp75fu4_93.png', 'S_ILSVRC2015_train_00638001.png', 'R_S_production_ID_3967181.png', 'R_S_yt-bTcbsBET7gc_270.png', 'R_S_ILSVRC2015_train_00599000.png', 'S

TypeError: object of type 'int' has no len()